In [2]:
import pandas as pd
import math
import os
import re
import random

import urllib, json
import plotly.express as px

In [89]:
def json_to_df_results(url):
    """ get flat pandas table from 'results' in json url"""
    
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())

    #normalize json into flat table
    df = pd.json_normalize(data['results'])
    
    return df

def json_to_df_meta(url):
    """ get flat pandas table from 'results' in json url"""
    
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())

    #normalize json into flat table
    df = pd.json_normalize(data['meta'])
    return df

def get_histogram(df,title):
    """ plot data in df as histogram (where x is 'term', y is 'count'), title must be string """
    
    fig = px.histogram(df, x='term',y='count', title=title)
    fig.update_layout(xaxis_title=None)
    
    return fig

def get_pie(df,title):
    """ plot data in df as pie chart (where x is 'term', y is 'count'), title must be string """
    fig = px.pie(df, values='count', names='term', title = title)
    return fig

def get_line(df, title):
    """ plot data in df as pie chart (where x is 'term', y is 'count'), title must be string """
    fig = px.line(df, y='count', x='term', title=title)
    return fig

def get_api_data(query,df_template):
    """Scrape data and construct df from API. Valid for reprter's type, country, seriousness of event and indication.
    
    Input:
    
    query -- API query (string)

    
    df_empty_template -- df, mandatory cols term, count, year_begin, year_end, event, drug_name, signal
    
    
    Returns: df with reporters type/country/seriousness of reaction per drug, all events, all years
    """

    
    try:
        df_json = json_to_df_results(query)

        df_json['year_begin'] = year_begin
        df_json['year_end'] = year_end

        df_json['event'] = event
        df_json['drug_name'] = drug_name
        df_json['signal'] = signal
    
        df_template = df_template.append(df_json)

        print(year_begin,year_end,event,drug_name,signal)

    except Exception as exception:
        print(exception)
        pass

    return df_template
    
def merge_with_reference(df,df_reference):
    
    """Replaces + with space in event name in df. Merges with reference df on term, requires df_reference with cols term, value and df with term, event.
        Drops 'term' col and replaces 'value' col name with 'term'to follow api format.
        Valid for reporter's type, seriousness of event and indication.
    """
    
    df['event']= df['event'].str.replace('+', ' ')
    df = df.merge(df_reference,how='left',on='term')
    df.drop(columns='term', inplace=True)
    df.rename(columns={'value':'term'},inplace=True)
    
    return df


def merge_with_country_code(df, country_codes):
    
    """Replaces + with space in event name in df. Merges with country_codes df on term, requires country_codes with cols term, value and df with term, event.
        Copies vals of country where where country - not code - reported. Drops 'term' col and replaces 'value' col name with 'term'to follow api format. 
        Valid for reporter's region.
    """
    df['event']= df['event'].str.replace('+', ' ')
    df = df.merge(country_codes,how='left',on='term')

    df['value'] = df['value'].fillna(df['term'])
    df.drop(columns='term', inplace=True)
    df.rename(columns={'value':'term'},inplace=True)
    
    return df

def get_api_data_age_sex(query, df_template):
    """Scrape data and construct df from API. Valid for age+sex group.
    
    Input:
    
    query -- API query (string)

    df_empty_template -- df, mandatory cols term, count, year_begin, year_end, event, drug_name, signal
    
    
    Returns: df with age group of reaction per drug, sex, all events, all years
    """

    try:
        df_json = json_to_df_results(query)
        df_json['sex'] = sex
        df_json['year_begin'] = year_begin
        df_json['year_end'] = year_end
        df_json['event'] = event
        df_json['drug_name'] = drug_name

        df_template = df_template.append(df_json)

        print(year_begin,year_end,sex,event,drug_name)

    except Exception as exception:
        print(exception)
        pass

    return df_template

def convert_sex(df, sex_code, sex_term):
    """
    Converts sex code to sex term, requires df with 'sex' col
    
    Input: 
    df -- pd dataframe, 'sex' col must be present
    sex_code (str) -- code for each sex
    sex_term (str) -- full term for each sex
    
    Returns:
    df -- modified pd dataframe
    
    """
    
    sex = df['sex'] == sex_code
    
    df.loc[sex,'sex'] = sex_term
    
    return df


def convert_age(df,age_format):
    
    """ Converts age (in months or days) to years. Requires df with 'term' (age). Rounds up converted age. 
    
    Input:
    df -- pd dataframe, col 'term' mandatory
    age_format (str) -- vals 'days' or 'months'
    
    Returns:
    df -- modified pd dataframe
    """
    if age_format == 'days':
        #where age in days, convert to years (assumed age <= 100yrs)
        df['term'].where(df['term'] < 1200, (df['term']/12/31), inplace=True) #keep as is where age <1200, otherwise convert days-->years
    
    if age_format == 'months':
        #where age in months, convert to years (assumed age <= 100 yrs)
        df['term'].where(df['term'] < 100 , (df['term']/12), inplace=True)  #keep as is where age <100, otherwise convert months-->years

    df['term'] = df['term'].astype(float).round(0)
    
    return df

def merge_and_clean(df,df_reference, col_to_merge_on):
    """
    Merges with reference df to get signal per event. Replaces 'term' with 'age'
    
    Input:
    df -- pd dataframe. col 'term' required
    df_reference -- pd dataframe to merge with
    col_to_merge_on (str) -- col name to merge on
    
    Returns:
    
    df -- modified pd dataframe
    
    """
    
    df[col_to_merge_on]= df[col_to_merge_on].str.replace('+', ' ')
    df = df.merge(df_reference,how='left', on='event')
    df.rename(columns={'term':'age'},inplace=True)
    
    return df

def get_api_data_signal_yearly(query,df_template):
    """Scrape data and construct df from API. Valid for signal yearly 
    
    Input:
    
    query -- API query (string)

    
    df_empty_template -- df, mandatory cols event, drug_name, signal
    
    
    Returns: df with drug, events, signal yearly
    """

    
    try:
        df_json = json_to_df_results(query)
        
        df_json['time'] = pd.to_datetime(df_json['time'], format='%Y%m%d')
        df_json = df_json.groupby(df_json['time'].dt.year)['count'].agg(['sum']) #counts get aggregated, sum is calculated

        df_json['event'] = event
        df_json['drug_name'] = drug_name
        df_json['signal'] = signal
    
        df_template = df_template.append(df_json)

        print(event,drug_name,signal)

    except Exception as exception:
        print(exception)
        pass

    return df_template

def rename_sum(df,col_to_rename_sum, col_to_rename_index):
    
    """ Final processing after count aggregation by sum. Renames to col_to_rename_sum. Drops old index, renames it by col_to_rename_index
    
        
    Input:
    df -- pd dataframe
    col_to_rename_sum (str) -- col name, will replace 'sum'
    col_to_rename_index (str) -- col name, will replace old index
    
    Returns:
    
    df -- modified pd dataframe
    
    
    """
    df.rename(columns={'sum':col_to_rename_sum}, inplace=True) 
    df.reset_index(inplace=True)
    df.rename(columns={'index':col_to_rename_index}, inplace=True) 
    
    return df

In [4]:
#dirs
# get required dirs for API querying output
demo = os.path.join(os.getcwd(), 'demo')
reporter_type = os.path.join('demo', 'reporter_type')
region = os.path.join('demo', 'region')
serious = os.path.join('demo', 'serious')
indication = os.path.join('demo', 'indication')
sex_age = os.path.join('demo', 'sex_age')

#create dirs if not existing
dirs = [demo,reporter_type,region,serious,indication,sex_age]
for directory in dirs:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [68]:
#read list of events + signal + drug_name
events = pd.read_excel('events.xlsx')
#remove whitespace
events['event'] = events['event'].str.strip()
#get df excerpt 
events2 = events[['event', 'signal']]

#replace space in composit event names with + (needed for api querying)
events['event'] = events['event'].str.replace(' ','+')
#needed for api querying
events['drug_name'] = events['drug_name'].str.upper()

#as in FDA db
sex_list = ['1','2']

#year range to query for 
years = pd.read_excel('years.xlsx')

#master df with necessary cols
df_master = pd.read_excel('df_empty_template.xlsx')
df_demo = pd.read_excel('demo.xlsx')

#df with reporter type vals & terms
reporter_type = pd.read_excel('reporter_type.xlsx')
#df with country codes
country_codes = pd.read_excel('country_codes.xlsx')
country_codes['value'] = country_codes['value'].str.upper()
#df with serious anotation
serious_vals = pd.read_excel('serious.xlsx')

#df for signal by year
df_signal_template = pd.read_excel('signal_df.xlsx')

In [38]:
#string "root" for openFDA API querying
query_reporter = 'https://api.fda.gov/drug/event.json?search=(receivedate:([{0}+TO+{1}]))+AND+(patient.reaction.reactionmeddrapt.exact:(%22{2}%22))+AND+(patient.drug.openfda.generic_name:%22{3}%22)&count=primarysource.qualification'
query_country = 'https://api.fda.gov/drug/event.json?search=(receivedate:([{0}+TO+{1}]))+AND+(patient.reaction.reactionmeddrapt.exact:(%22{2}%22))+AND+(patient.drug.openfda.generic_name:%22{3}%22)&count=primarysource.reportercountry.exact'
query_serious = 'https://api.fda.gov/drug/event.json?search=(receivedate:([{0}+TO+{1}]))+AND+(patient.reaction.reactionmeddrapt.exact:(%22{2}%22))+AND+(patient.drug.openfda.generic_name:%22{3}%22)&count=serious'
query_indi = 'https://api.fda.gov/drug/event.json?search=(receivedate:([{0}+TO+{1}]))+AND+(patient.reaction.reactionmeddrapt.exact:(%22{2}%22))+AND+(patient.drug.openfda.generic_name:%22{3}%22)&count=patient.drug.drugindication.exact'
query_age_sex = 'https://api.fda.gov/drug/event.json?search=(receivedate:([{0}+TO+{1}])+AND+patient.patientsex:{2})+AND+(patient.reaction.reactionmeddrapt.exact:(%22{3}%22))+AND+(patient.drug.openfda.generic_name:%22{4}%22)&count=patient.patientonsetage'
query_signal = 'https://api.fda.gov/drug/event.json?search=(patient.reaction.reactionmeddrapt.exact:(%22{0}%22))+AND+(patient.drug.openfda.generic_name:%22{1}%22)&count=receivedate'

In [16]:
#reporter type 
df_reporter = pd.DataFrame(columns=df_master.columns)
for event,drug_name,signal in zip(events.event, events.drug_name, events.signal):

    for year_begin, year_end in zip(years.year_begin,years.year_end):
        
        query = query_reporter.format(year_begin,year_end,event,drug_name)
 
        df_reporter = get_api_data(query, df_reporter)
        
df_reporter = merge_with_reference(df_reporter,reporter_type)
df_reporter.to_excel('{0}_reporter_type.xlsx'.format(drug_name))

In [21]:
#reporter's region
df_country = pd.DataFrame(columns=df_master.columns)

for event,drug_name,signal in zip(events.event, events.drug_name, events.signal):

    for year_begin, year_end in zip(years.year_begin,years.year_end):
        
        query = query_country.format(year_begin,year_end,event,drug_name)
 
        df_country = get_api_data(query, df_country)
        
df_country = merge_with_country_code(df_country, country_codes)
df_country.to_excel('{0}_reporter_country.xlsx'.format(drug_name))

In [26]:
#serious
df_serious = pd.DataFrame(columns=df_master.columns)

for event,drug_name,signal in zip(events.event, events.drug_name, events.signal):

    for year_begin, year_end in zip(years.year_begin,years.year_end):
        
        query = query_serious.format(year_begin,year_end,event,drug_name)
 
        df_serious = get_api_data(query, df_serious)
        
df_serious = merge_with_reference(df_serious,serious_vals)
df_serious.to_excel('{0}_reaction_seriousness.xlsx'.format(drug_name))

In [29]:
#indication
df_indication = pd.DataFrame(columns=df_master.columns)

for event,drug_name,signal in zip(events.event, events.drug_name, events.signal):

    for year_begin, year_end in zip(years.year_begin,years.year_end):
        
        query = query_indi.format(year_begin,year_end,event,drug_name)
 
        df_indication = get_api_data(query, df_indication)
        
df_indication['event']= df_indication['event'].str.replace('+', ' ')
df_indication.to_excel('{0}_indication.xlsx'.format(drug_name))

In [66]:
#age_sex group
demo_df = pd.DataFrame(columns=df_demo.columns)
for event,drug_name in zip(events.event, events.drug_name): #no 'signal' here, later on merged to get 'signal'
    for sex in sex_list:

        for year_begin, year_end in zip(years.year_begin,years.year_end):

            query = query_age_sex.format(year_begin,year_end,sex,event,drug_name)

            demo_df = get_api_data_age_sex(query, demo_df)
            
            
#convert age in months/days to years 
#attention: run days conversion always first!
demo_df = convert_age(demo_df, 'days')
demo_df = convert_age(demo_df, 'months')

#convert each sex codes to sex term
demo_df = convert_sex(demo_df, '2', 'female')
demo_df = convert_sex(demo_df, '1', 'male')

#merge to get 'signal', rename 'term' col
demo_df = merge_and_clean(demo_df,events2,'event')
demo_df.to_excel('{0}_Age_Sex.xlsx'.format(drug_name))

In [96]:
#events,drugs,signals yearly
signal_df = pd.DataFrame(columns=df_signal_template.columns)

for event,drug_name,signal in zip(events.event, events.drug_name, events.signal):

    query = query_signal.format(event,drug_name)
    signal_df = get_api_data_signal_yearly(query, signal_df)
    
#final clean up 
signal_df = rename_sum(signal_df,'count', 'year') #sum = count (follow fda api naming convention), old index = year
signal_df.to_excel('{0}_signal_yearly.xlsx'.format(drug_name), index=False)

In [ ]:
# check 100 most common drugs, all our five included there
# most_common_drugs = json_to_df_results('https://api.fda.gov/drug/event.json?&count=patient.drug.openfda.generic_name.exact')
# most_common_drugs.to_excel('most_common_drugs.xlsx')